In [237]:
import boto3
import yaml

session = boto3.Session(region_name='us-east-2')
ec2 = session.client(service_name='ec2')

with open('config.yaml', 'r') as f:
    yaml_file = f.read()

yaml_config = yaml.load(yaml_file, Loader=yaml.FullLoader)

user_name = yaml_config['users'][0]['login']

# Add Volumes
# Following block of code will create shell script to run at start of VM initialization
# It will mount drive to location given in cofig with given file system type

shell_cmd_base = """sudo mkfs -t {0} /dev/nvme{1}n1 && sudo mkdir -p {2} && sudo mount /dev/nvme{1}n1 {2}"""

shell_script = "#!/bin/bash \n"

volumes = yaml_config['volumes']

block_device_mapping = []
for i, vol in enumerate(volumes):
    device = dict()
    device['DeviceName'] = vol['device']
    device['VirtualName'] = vol['device']
    device['Ebs'] = {
        'VolumeSize': vol['size_gb'],
        'VolumeType': 'gp2'
    }
    block_device_mapping.append(device)
    shell_script += shell_cmd.format(vol['type'], i+1, vol['mount'])
    shell_script += '\n'

### create security group

sec_grp = ec2.create_security_group(Description='ec2 ssh {0}'.format(user_name),GroupName='ec2_ssh_{0}'.format(user_name))

sec_res = ec2.authorize_security_group_ingress(
    GroupId=sec_grp['GroupId'],
    FromPort=22,
    ToPort=22,
    CidrIp='0.0.0.0/0',
    IpProtocol='tcp'
)

### import key pair

key_res = ec2.import_key_pair(KeyName=user_name, PublicKeyMaterial=yaml_config['users'][0]['ssh_key'].encode())

### create ec2 instance

ec2_res = ec2.run_instances(
    InstanceType=yaml_config['instance_type'],
    BlockDeviceMappings=block_device_mapping,
    ImageId = 'ami-013de1b045799b282',
    MinCount = 1,
    MaxCount = 1,
    KeyName=user_name,
    UserData=shell_script
)